In [1]:
import pandas as pd
import time
from math import isclose

from hccpy.hcc import HCCEngine
from risk_adjustment_model import MedicareModelV24, MedicareModelV28

he_v28 = HCCEngine("28")
he_v24 = HCCEngine("24", norm_params={"C": 1.118})
ra_v24 = MedicareModelV24(year=2022) #hccpy looks like it has only loaded up to 2022
ra_v28 = MedicareModelV28()

df_demo = pd.read_csv('test_demo_dataset_10k.txt', sep='|')
df_diag = pd.read_csv('test_dataset_10k.txt', sep='|')

demographics = df_demo.set_index('member_id')
diagnoses = df_diag.set_index('member_id')

In [2]:
start_time = time.time()

hcc_v24_results_dict = {}
hcc_v28_results_dict = {}
model_v24_results_dict = {}
model_v28_results_dict = {}

# loop over people
#--------------------------------------------------------------------
for row in demographics.itertuples():

    # put input demographic data into local variables
    #--------------------------------------------------------------------
    member_id = row.Index
    gender = row.gender
    ltimcaid = row.medicaid
    orec = row.orec
    age = row.age

    if member_id in diagnoses.index:
        if len(diagnoses.loc[member_id]) == 1:
            diagnosis_for_pt = list(diagnoses.loc[member_id]['dx_code'])
        else:
            diagnoses_for_pt = diagnoses.loc[member_id]['dx_code'].to_list()
    else:
        diagnoses_for_pt = []

    for elig in ["CNA", "CND", "CPA", "CPD", "CFA", "CFD", "NE", "INS"]:
        results_hcc_v24 = he_v24.profile(diagnoses_for_pt, age=age, sex=gender, elig=elig, orec=orec, medicaid=ltimcaid)
        results_hcc_v28 = he_v28.profile(diagnoses_for_pt, age=age, sex=gender, elig=elig, orec=orec, medicaid=ltimcaid)
        results_v24 = ra_v24.score(gender=gender, orec=orec, medicaid=ltimcaid, diagnosis_codes=diagnoses_for_pt, age=age, population=elig)
        results_v28 = ra_v28.score(gender=gender, orec=orec, medicaid=ltimcaid, diagnosis_codes=diagnoses_for_pt, age=age, population=elig)
    
        if not isclose(results_hcc_v24['risk_score'], results_v24.score_raw):
            hcc_v24_results_dict[elig+"|"+member_id] = results_hcc_v24
            model_v24_results_dict[elig+"|"+member_id] = results_v24
    
        if not isclose(results_hcc_v28['risk_score'], results_v28.score_raw):
            hcc_v28_results_dict[elig+"|"+member_id] = results_hcc_v28
            model_v28_results_dict[elig+"|"+member_id] = results_v28

end_time = time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time)

Execution time: 142.35359907150269


In [4]:
model_v28_results_dict

{}

In [25]:
hcc_v24_results_dict['INS|a2c8e158-2518-40c3-b61e-6a368062a762']['hcc_lst']


['HCC40',
 'HCC72',
 'HCC173',
 'HCC35',
 'HCC167',
 'HCC55',
 'HCC170',
 'HCC188',
 'HCC75',
 'HCC59',
 'HCC2',
 'HCC18',
 'HCC39',
 'HCC10',
 'HCC100',
 'HCC189',
 'D10P',
 'gSubstanceUseDisorder_gPsych',
 'SEPSIS_ARTIF_OPENINGS']

In [26]:
model_v24_results_dict['INS|a2c8e158-2518-40c3-b61e-6a368062a762'].category_list

['F65_69',
 'HCC40',
 'HCC72',
 'HCC173',
 'HCC35',
 'HCC167',
 'HCC170',
 'HCC55',
 'HCC188',
 'HCC75',
 'HCC59',
 'HCC2',
 'HCC18',
 'HCC39',
 'HCC10',
 'HCC100',
 'HCC189',
 'gSubstanceUseDisorder_gPsych',
 'SEPSIS_ARTIF_OPENINGS',
 'D10P']

In [27]:
model_v24_results_dict['INS|a2c8e158-2518-40c3-b61e-6a368062a762'].category_details

{'F65_69': {'coefficient': 1.245, 'diagnosis_map': None},
 'HCC40': {'coefficient': 0.292,
  'diagnosis_map': ['M06842', 'M05512', 'M488X7', 'M3214', 'M06031']},
 'HCC72': {'coefficient': 0.289, 'diagnosis_map': ['S14157S']},
 'HCC173': {'coefficient': 0.092, 'diagnosis_map': ['S58119A']},
 'HCC35': {'coefficient': 0.355, 'diagnosis_map': ['K51311']},
 'HCC167': {'coefficient': 0.0, 'diagnosis_map': ['S02112S']},
 'HCC170': {'coefficient': 0.0, 'diagnosis_map': ['S72436C']},
 'HCC55': {'coefficient': 0.178, 'diagnosis_map': ['F1114', 'F13988']},
 'HCC188': {'coefficient': 0.514, 'diagnosis_map': ['K9411']},
 'HCC75': {'coefficient': 0.332, 'diagnosis_map': ['M05512']},
 'HCC59': {'coefficient': 0.187, 'diagnosis_map': ['T592X2S']},
 'HCC2': {'coefficient': 0.324, 'diagnosis_map': ['P0270', 'R6510']},
 'HCC18': {'coefficient': 0.44, 'diagnosis_map': ['E0943']},
 'HCC39': {'coefficient': 0.401, 'diagnosis_map': ['M02839']},
 'HCC10': {'coefficient': 0.461, 'diagnosis_map': ['C8286', 'C81

In [28]:
hcc_v24_results_dict['INS|a2c8e158-2518-40c3-b61e-6a368062a762']

{'risk_score': 6.113,
 'risk_score_age': 1.306,
 'risk_score_adj': 5.1452,
 'risk_score_age_adj': 1.0992,
 'details': {'INS_F65_69': 1.245,
  'INS_LTIMCAID': 0.061,
  'INS_HCC40': 0.292,
  'INS_HCC72': 0.289,
  'INS_HCC173': 0.092,
  'INS_HCC35': 0.355,
  'INS_HCC167': 0.0,
  'INS_HCC55': 0.178,
  'INS_HCC170': 0.0,
  'INS_HCC188': 0.514,
  'INS_HCC75': 0.332,
  'INS_HCC59': 0.187,
  'INS_HCC2': 0.324,
  'INS_HCC18': 0.44,
  'INS_HCC39': 0.401,
  'INS_HCC10': 0.461,
  'INS_HCC100': 0.111,
  'INS_HCC189': 0.357,
  'INS_D10P': 0.0,
  'INS_gSubstanceUseDisorder_gPsych': 0.0,
  'INS_SEPSIS_ARTIF_OPENINGS': 0.474},
 'hcc_lst': ['HCC40',
  'HCC72',
  'HCC173',
  'HCC35',
  'HCC167',
  'HCC55',
  'HCC170',
  'HCC188',
  'HCC75',
  'HCC59',
  'HCC2',
  'HCC18',
  'HCC39',
  'HCC10',
  'HCC100',
  'HCC189',
  'D10P',
  'gSubstanceUseDisorder_gPsych',
  'SEPSIS_ARTIF_OPENINGS'],
 'hcc_map': {'C8286': ['HCC10'],
  'F1114': ['HCC55'],
  'R6510': ['HCC2'],
  'S14157S': ['HCC72'],
  'M06842': ['HCC4

In [6]:
results_v24

ScoringResults(gender='F', orec=1, medicaid=True, age=85, dob=None, diagnosis_codes=['S14157S', 'C8286', 'M06842', 'M05512', 'M488X7'], year=2022, population='INS', risk_model_age=85, risk_model_population='INS', model_version='v24', model_year=2022, coding_intensity_adjuster=0.941, normalization_factor=1.118, score_raw=2.233, disease_score_raw=1.374, demographic_score_raw=0.859, score=1.8795, disease_score=1.1564, demographic_score=0.723, category_list=['F85_89', 'LTIMCAID', 'HCC75', 'HCC40', 'HCC10', 'HCC72', 'D4'], category_details={'F85_89': {'coefficient': 0.798, 'diagnosis_map': None}, 'LTIMCAID': {'coefficient': 0.061, 'diagnosis_map': None}, 'HCC75': {'coefficient': 0.332, 'diagnosis_map': ['M05512']}, 'HCC40': {'coefficient': 0.292, 'diagnosis_map': ['M06842', 'M05512', 'M488X7']}, 'HCC10': {'coefficient': 0.461, 'diagnosis_map': ['C8286']}, 'HCC72': {'coefficient': 0.289, 'diagnosis_map': ['S14157S']}, 'D4': {'coefficient': 0.0, 'diagnosis_map': None}})

In [ ]:
unique_disease_cats

In [ ]:

dx_codes = ['S14157S', 'C8286', 'M06842', 'M05512', 'M488X7', 'P0270', 'C50922', 'K51311', 'M3214', 'S72436C', 'E0943', 
            'C8175', 'S068AAS', 'F1114', 'M06031', 'M02839',
            'S02112S', 'C9590', 'K9411', 'I6330', 'R6510', 'S98912D', 'T592X2S', 'F13988', 'S58119A']

rp_v28 = he_v28.profile(dx_codes,
                    age=65, sex="F", elig='NE') 
# rp_v24 = he_v24.profile(bene_obj.diagnosis_codes,
#                     age=bene_obj.age, sex="F", elig="CND") 

In [ ]:
rp_v28

In [ ]:
def _get_demographic_cats(age, gender, population):
        """
        Determine the demographic category based on age and gender.

        Args:
            age (int): The age of the individual.
            gender (str): The gender of the individual ('Male', 'Female', etc.).

        Returns:
            str: The demographic category of the individual.

        Notes:
            This function determines the demographic category of an individual based on their age and gender. 
            It assigns individuals to predefined demographic categories, such as age and gender bands, 
            defined as 'F0_34', 'M35_44', etc. The categories are hard-coded within the function.
            
            The function iterates through the predefined demographic categories and checks if the provided 
            age falls within the range specified for each category. Once a matching category is found, 
            it returns that category.
        """
        # PF: Might want to refactor to use yaml/json with demographic type to get the categories
        if population[:2] == 'NE':
            demo_category_ranges = [
                '0_34', '35_44', '45_54', '55_59', '60_64',
                '65', '66', '67', '68', '69', '70_74', 
                '75_79', '80_84', '85_89', '90_94', '95_GT',
            ]
        else:
            demo_category_ranges = [
                '0_34', '35_44', '45_54', '55_59', '60_64',
                '65_69', '70_74', '75_79', '80_84', '85_89', 
                '90_94', '95_GT',
            ]
        
        for age_range in demo_category_ranges:
            age_band = age_range.replace(gender, '').split('_') 
            lower, upper = 0, 999
            if len(age_band) == 1:
                lower = int(age_band[0])
                upper = lower + 1
            elif age_band[1] == 'GT':
                lower = int(age_band[0])
            else: 
                lower = int(age_band[0])
                upper = int(age_band[1]) + 1
            if lower <= age < upper:
                demographic_category_range = age_range
                break

        if population[:2] == 'NE':
            demographic_category = f'NE{gender}{demographic_category_range}'
        else:
            demographic_category = f'{gender}{demographic_category_range}'

        return demographic_category

In [ ]:
cat = _get_demographic_cats(65, 'F', 'NE_MCAID_ORIGDIS')

In [ ]:
cat